In [4]:
pip install unidecode beautifulsoup4 python-magic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [python-magic]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import base64
coded_string = '''Q5YACgA...'''
# Add padding to the Base64 string if necessary
missing_padding = len(coded_string) % 4
if missing_padding:
	coded_string += '=' * (4 - missing_padding)

# Decode the Base64 string
decoded_data = base64.b64decode(coded_string)
decoded_data


b'C\x96\x00\n\x00'

In [3]:
def decode_hexstring(hexstring):
    decoded = ''

    for i in range(0, len(hexstring), 2):
        b = hexstring[i:i+2]
        b = b.decode() # it's a byte-string

        try:
            c = bytes.fromhex(b).decode()
        except: # the last char might be missing
            c = '☐'

        decoded = decoded + c

    return decoded

print(decode_hexstring(b'737030306b792d686578737472696e676'))


sp00ky-hexstring☐


In [6]:
import base64
import codecs
import re
import html
import unicodedata
from bs4 import BeautifulSoup
from unidecode import unidecode


# ============================
# 0. Remove zero-width
# ============================
def remove_zero_width(text):
    zero_width = [
        '\u200b', '\u200c', '\u200d',
        '\u2060', '\u180e', '\ufeff'
    ]
    for z in zero_width:
        text = text.replace(z, '')
    return text


# ============================
# 1. Unicode normalization
# ============================
def normalize_unicode(text):
    return unicodedata.normalize("NFKC", text)


# ============================
# 2. Strip HTML/Markdown
# ============================
def strip_html_markdown(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    text = re.sub(r"`([^`]+)`", r"\1", text)
    text = re.sub(r"[*_#>\[\]\(\)]", "", text)
    return html.unescape(text)


# ============================
# 3. Decode Base64 safely
# ============================
def try_decode_base64(text):
    try:
        decoded = base64.b64decode(text).decode("utf-8")
        if len(decoded) > 0:
            return decoded
    except:
        pass
    return text


# ============================
# 4. Decode Hex
# ============================
def try_decode_hex(text):
    try:
        cleaned = text.replace(" ", "")
        return bytes.fromhex(cleaned).decode("utf-8")
    except:
        return text


# ============================
# 5. Decode rot13
# ============================
def try_rot13(text):
    try:
        return codecs.decode(text, "rot_13")
    except:
        return text


# ============================
# 6. Fix repeated chars
# ============================
def remove_repeated_chars(text):
    return re.sub(r"(.)\1{2,}", r"\1", text)


# ============================
# 7. Fix typos / scrambled words
# ============================
COMMON_WORDS = {
    "ignore": ["ignroe", "ign0re", "ignr0e", "ig nore"],
    "system": ["sysetm", "systme", "sys tem"],
    "rules": ["ruels", "rul3s"],
    "prompt": ["promtp", "pr0mpt"],
    "instructions": ["insturctions"]
}

def fix_scrambled_words(text):
    for correct, variations in COMMON_WORDS.items():
        for v in variations:
            text = re.sub(v, correct, text, flags=re.IGNORECASE)
    return text


# ============================
# 8. Normalize mixed-script
# ============================
def normalize_mixed_script(text):
    # fix Arabic letter variations
    arabic_map = {
        "ى": "ي",
        "ئ": "ي",
        "ة": "ه",
    }
    for k, v in arabic_map.items():
        text = text.replace(k, v)

    # detect mixed script and collapse
    pattern = r"[A-Za-z]+[\u0600-\u06FF]+|[\u0600-\u06FF]+[A-Za-z]+"
    text = re.sub(pattern, lambda m: unidecode(m.group()), text)

    return text


# ============================
# FINAL — Full Normalization Pipeline
# ============================
def normalize_input(text):

    # 0 — zero-width
    text = remove_zero_width(text)

    # 1 — unicode
    text = normalize_unicode(text)

    # 2 — HTML/Markdown
    text = strip_html_markdown(text)

    # 3 — decoding attempts
    for decoder in [try_decode_base64, try_decode_hex, try_rot13]:
        decoded = decoder(text)
        if decoded != text:
            text = decoded

    # 4 — repeated chars
    text = remove_repeated_chars(text)

    # 5 — fix scrambled words
    text = fix_scrambled_words(text)

    # 6 — mixed script normalization
    text = normalize_mixed_script(text)

    # 7 — lowercase final
    text = text.lower()

    return text
